In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os
from bs4 import BeautifulSoup
import re

import nltk.corpus
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from numpy.linalg import norm
from sklearn.metrics.pairwise import linear_kernel

import time
import webbrowser 

from multiprocessing.pool import ThreadPool as Pool
from multiprocessing import *


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
os.chdir(r"C:\Users\swani\Python Projects\Case Bucket\Data")

In [3]:
data = pd.read_excel('N-Grams Data.xlsx')
length = int(len(data))
print('Data Length: %d' %(length))
print(data.head(5))

df = pd.read_excel('ALl-keywords-2021.xlsx')
exceptions = df["Keywords Flagged by IT"].tolist()

Data Length: 107161
   Unnamed: 0      Name  Article Number                          N-Grams
0           0  20467935          538672            vhdl get csgmss error
1           1  20467935          538672            get csgmss error data
2           2  20467935          538672           csgmss error data type
3           3  20467935          538672       error data type unresolved
4           4  20467935          538672  data type unresolved std_ulogic


In [3]:
start = time.time()
a = input()
a = a.lower()
a = a.replace('[^\w\s]','')
stop_words = stopwords.words('english')
a = [word for word in a.split() if word not in (stop_words)]
a = ' '.join(a)
a = [word for word in a.split() if word not in (exceptions)]
a = ' '.join(a)
lmtzr = WordNetLemmatizer()
lemmatized = [lmtzr.lemmatize(word) for word in word_tokenize(a)]
a = ' '.join(lemmatized)
print('')
print('Processed input: %s' %(a))

ngrams = data['N-Grams']
ngrams_list = ngrams.tolist()
ngrams_list.append(a)

vectorizer = TfidfVectorizer().fit_transform(ngrams_list)
input_vector = vectorizer[-1]
vectorizer = vectorizer[:-1]

cosine_similarities = linear_kernel(vectorizer[:], input_vector).flatten()

sort_cs = np.sort(cosine_similarities)[::-1]
top3_values = sort_cs[:3]
if (any(i >= 0.8 for i in top3_values) is True):
    print(top3_values)
else:
    top3_values = [0,0,0]
print('')
print("Cosine Similarities: ",top3_values)

indices_list = []

if (all(i == 0 for i in top3_values) is True):
    indices_list = [0,0,0]
else:
    for value in top3_values:
        max_list = [i for i,val in enumerate(cosine_similarities) if val==value]
        indices_list.append(max_list[0])

print('')
print("Indices of 3 maximum value: ",indices_list)

if (indices_list == [0,0,0]):
    print('')
    print('The KB Articles: None')
    KB_Articles = []
else:
    lst = [data.iloc[i]['Name'] for i in indices_list]
    KB_Articles = np.unique(lst)
    print('')
    print('The KB Articles: %a'%KB_Articles)
    unique_indices = [lst.index(i) for i in KB_Articles]

print('')
print('Output (Similar N-Grams): ')
op_ngrams = []
if (indices_list == [0,0,0]):
    print("No N-grams match the input query")
else:
    for i in unique_indices:
        index = indices_list[i]
        op_ngrams.append(ngrams_list[index])
    print(op_ngrams)

end = time.time()
print('')
print('Time Taken to Respond: %d seconds' %(end-start))

for article in KB_Articles:
    str = 'http://cdsinfo.cadence.com/cgi-bin/cdsinfoprod?input={}&type=_&codmode=p'.format(article)
    webbrowser.open_new(str) 

NameError: name 'time' is not defined

In [ ]:
A_sparse = sparse.csr_matrix(vectorizer)

similarities = cosine_similarity(A_sparse)
print('pairwise dense output:\n {}\n'.format(similarities))

#also can output sparse matrices
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

A_sparse = sparse.csr_matrix(vectorizer)

NameError: name 'vectorizer' is not defined